In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

In [ ]:
df = pd.read_csv('/content/spam.csv', delimiter=',', encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
X = df.v2
Y = df.v1
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)
Y = Y.reshape(-1,1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
tokenizer = Tokenizer(num_words=2000, lower=True)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences, maxlen=200)

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(2000, 50, input_length=200))
model.add(LSTM(64))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 50)           100000    
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 146,337
Trainable params: 146,337
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
28/28 [==============================] - 14s 394ms/step - loss: 0.3208 - accuracy: 0.8825 - val_loss: 0.1635 - val_accuracy: 0.9552
Epoch 2/10
28/28 [==============================] - 12s 418ms/step - loss: 0.0922 - accuracy: 0.9787 - val_loss: 0.0703 - val_accuracy: 0.9798
Epoch 3/10
28/28 [==============================] - 10s 368ms/step - loss: 0.0391 - accuracy: 0.9907 - val_loss: 0.0597 - val_accuracy: 0.9809
Epoch 4/10
28/28 [==============================] - 10s 371ms/step - loss: 0.0283 - accuracy: 0.9921 - val_loss: 0.0507 - val_accuracy: 0.9843
Epoch 5/10
28/28 [==============================] - 10s 371ms/step - loss: 0.0192 - accuracy: 0.9941 - val_loss: 0.0501 - val_accuracy: 0.9832
Epoch 6/10
28/28 [==============================] - 12s 418ms/step - loss: 0.0123 - accuracy: 0.9966 - val_loss: 0.0624 - val_accuracy: 0.9854
Epoch 7/10
28/28 [==============================] - 10s 374ms/step - loss: 0.0096 - accuracy: 0.9978 - val_loss: 0.0493 - val_accuracy: 0.9877

In [ ]:
model.save("model.h5")

In [ ]:
test_sequences = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(test_sequences, maxlen=200)

In [ ]:
acc = model.evaluate(X_test, y_test)

35/35 [==============================] - 1s 31ms/step - loss: 0.0594 - accuracy: 0.9928


In [ ]:
def predict(message):
    txt = tokenizer.texts_to_sequences(message)
    txt = sequence.pad_sequences(txt, maxlen=200)
    preds = model.predict(txt)
    if preds > 0.5:
        print("Spam")
    else:
        print("Not Spam")

In [ ]:
predict(["Sorry, I'll call after the meeting."])

1/1 [==============================] - 1s 558ms/step
Not Spam


In [ ]:
predict(["Congratulations!!! You won $50,000. Send message LUCKY100 to XXXXXXXXXX to recieve your prize."])

1/1 [==============================] - 0s 34ms/step
Spam
